
### 这一节我们主要学习

+ 序列标签
+ 隐马尔科夫模型
+ 条件随机场
+ 结构感知器和结构化支持向量机




### 1. 序列标签

输入为一个序列，输出为另一个序列。不仅 RNN 可以解决这个问题，同时基于结构化学习也能解决这个问题。

![](http://imgbed.momodel.cn/35-1 seq label.png)

#### 1.1 任务——词性标注

+ 用词性注释句子中的每个单词

![](http://imgbed.momodel.cn/35-2 annotate.png)
+ 对后续的句法分析和词义消歧等有用。

![](http://imgbed.momodel.cn/35-3 disambiguation.png)

### 2. 隐马尔科夫模型

+ 如何产生一个句子
    + 第一步：根据语法，产生词性序列
    + 第二步：根据字典，有给出的词性序列产生一个句子

![](http://imgbed.momodel.cn/35-4 hmm.png)



![](http://imgbed.momodel.cn/35-5 step1.png)





![](http://imgbed.momodel.cn/35-6 step2.png)

如何计算 $P(x,y)=P(x)P(y|x)$?


计算过程如下：

![](http://imgbed.momodel.cn/35-7 calculate.png)


![](http://imgbed.momodel.cn/35-8 step12.png)
第一步计算转换概率，第二步计算发射概率。


+ 估算概率
  + 如何计算$P(V|PN),P(saw|V)$?
  + 从训练数据中获得

![](http://imgbed.momodel.cn/35-9 training data.png)

使用训练样本中事件发生的频率来估计概率：

![](http://imgbed.momodel.cn/35-10 estimation.png)


有了各个事件发生的概率之后，我们就可以求出可能性最大的 $y$

![](http://imgbed.momodel.cn/35-11 compute P.png)

如果枚举所有的 $y$，计算的复杂度会非常的大；

这里我们使用[维比特算法](https://blog.csdn.net/gzmfxy/article/details/78712878)来求解可能性最大的 $y$

![](http://imgbed.momodel.cn/35-12 enumerate.png)



![](http://imgbed.momodel.cn/35-13 summary.png)


![](http://imgbed.momodel.cn/35-15 high probability.png)

按照训练集的数据来看，$y_l$ 最可能应该是 $D$，但是 HMM 模型可能会自行“脑补“出 $V$.


![](http://imgbed.momodel.cn/35-16 little.png)
 + **优点：**
     + 当训练数据较少的时候，使用 HMM 模型比较好。

 + **缺点：**
     + HMM 模型可能产生样本中并未出现的情况；使用 CRF 模型可以解决该问题。
     + HMM 模型并不能保证数据集中数据点发生的概率是最大的





### 3. 条件随机场

#### 3.1 CRF


$P(x,y)∝ exp(w \cdot \phi(x,y))$

+ $\phi(x,y)$ 是特征向量
+ $w$ 是通过训练数据学习出来的权重向量
+ $exp(w \cdot \phi(x,y))$ 为正，有可能大于 1


![](http://imgbed.momodel.cn/35-17 crf.png)


![](http://imgbed.momodel.cn/35-18 log.png)
将 $P(x,y)$ 两边同取 $log$ 变换


![](http://imgbed.momodel.cn/35-19 aggagag.png)
将 $\sum logP(x_l|y_l)$ 表示成 $\sum logP(t|s)*N{s,t}(x,y)$ 的形式



![](http://imgbed.momodel.cn/35-20 example.png)

同理，对于 $logP(x,y)$ 可以得到如下的结果：


![](http://imgbed.momodel.cn/35-21 logpxy.png)

将上式表示成特征向量乘以权重向量的形式：

![](http://imgbed.momodel.cn/35-22 frame.png)
**注意：**

![](http://imgbed.momodel.cn/35-23 improve.png)
由于我们在训练的时候没有给$w$任何的限制，是有正有负的，我们将 $=$ 改为 $ ∝$（成正比）。

#### 3.2 特征向量

$\phi (x,y)$ 有两部分：

+ 标签和单词之间的关系

该部分的维度可能比较大，但是非零的值比较少，较为稀疏

![](http://imgbed.momodel.cn/35-24 features factor.png)

+ 标签之间的关系

该部分的维度和标签的个数相关

![](http://imgbed.momodel.cn/35-26 partt2.png)
** 注：$\phi (x,y)$ 可以定义为任意你想要的**

#### 3.3 训练标准




![](http://imgbed.momodel.cn/35-27 train criteria.png)
我们需要增大我们观察到的事件的概率，减小未观察到的事件的概率。

#### 3.4 梯度上升


![](http://imgbed.momodel.cn/35-28 gradient ascent.png)

#### 3.5 训练



![](http://imgbed.momodel.cn/35-29 ascent.png)
+ 第一项表示（绿下划线）：如果单词被标签s标记在训练集中，那么就要增大 $w_{s,t}$
+ 第二项表示（黄下划线）：如果单词被标签s标记不在训练集中，那么就要减小 $w_{s,t}$


![](http://imgbed.momodel.cn/35-30 equivalent.png)

**随机梯度下降:**

随机选择一个数据$(x^n,y^n)$，进行训练：


![](http://imgbed.momodel.cn/35-31 sochastic.png)

#### 3.6 推理




用维特比算法求出最大的y。

![](http://imgbed.momodel.cn/35-32 vtebi.png)

#### 3.6 CRF v.s. HMM


 

![](http://imgbed.momodel.cn/35-33 vs.png)

相比于 HMM，CRF 除了增大正确事件发生的概率外，同时减小了错误事件发生的概率（HMM 就没有减小）；

因此，CRF 通常可以产生更好的结果。


![](http://imgbed.momodel.cn/35-34 synthetic.png)


![](http://imgbed.momodel.cn/35-35 vs22.png)
当$\alpha$越小时，实验结果越差；

当$\alpha > \frac {1}{2}$时，HMM的表现更好；

当$\alpha < \frac{1}{2}$时，CRF的表现更好。

#### 3.7 总结




![](http://imgbed.momodel.cn/35-36 summary.png)

<br>

### 4.结构感知器和支持向量机

#### 4.1 结构感知器



结构感知机的模型如图所示，

![](http://imgbed.momodel.cn/35-37 structured perceptron.png)


![](http://imgbed.momodel.cn/35-38  perceptron123.png)
不同的地方在于结构感知机减去的几率最大的 y 形成的 $\phi$；

CRF减去的是所有可能的 y 形成的 $\phi$ 乘上发生这个 y 的概率。


#### 4.2 结构化支持向量机 



![](http://imgbed.momodel.cn/35-39 ssvm.png)
在步骤三中，与结构感知器不同的是，结构化支持向量机需要考虑边距和误差，方法有：

   + 梯度下降法
   + 二次规划（切割平面法）


![](http://imgbed.momodel.cn/35-40 error.png)
+ $\Delta(\hat y,y)$：y 与 $\hat y$ 的偏差

+ 结构化支持向量机的损失函数是误差函数的上界

+ 理论上，误差函数可以设置为任何你喜欢的样子

+ 在设置误差函数时，要保证能够 <u>Problem $2.1$</u> 能够方便地解决。
    + Problem $2.1$ : $\hat y^{n}$ = argmax[$\Delta(\hat y,y)+w \cdot \phi(x^{n},y)$]（可以用 Vitervi 算法解决）

不同方法的实验结果：

![](http://imgbed.momodel.cn/35-41 perfomance.png)



![](http://imgbed.momodel.cn/35-42 rnn.png)
+ CRF，结构化支持向量机等的优点：
  + 考虑整个序列
  + 可以明确考虑标签依赖性
  + 损失函数是误差的上界
+ RNN 的优点：
  + 网络结构比较深
  
但是，RNN 的优点相比于 CRF，结构化支持向量机等的优点更好，所以 RNN 表现得过更好。




![](http://imgbed.momodel.cn/35-43 integrate.png)

+ 语音识别： CNN/LSTM/DNN + HMM


![](http://imgbed.momodel.cn/35-45 hmm integarte.png)
用贝叶斯公式求解概率 $P(x_l|y_l)$

+ 语义标记：Bi-directional LSTM + CRF/Structured SVM

![](http://imgbed.momodel.cn/35-46 crf tagggg.png)
利用 RNN 模型得到特征向量，再用 CRF 进行标记。

### 5.总结



![](http://imgbed.momodel.cn/35-47 remark.png)
这些方法都能加上深度学习，从而有一个更好的表现